<a href="https://www.kaggle.com/code/masoudnaghshbandi/titanic-machine-learning-from-disaster?scriptVersionId=108146419" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


A bout data set, it has 11 columns,and most of them is clear what they are just a bit information about Embark which implies where the traveler mounted from. There are three possible values for Embark — Southampton, Cherbourg, and Queenstown.
parch: The dataset defines family relations in this way
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

# Import Libraries

In [69]:
from sklearn.model_selection import  train_test_split
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

# Load train and test data get into it

In [70]:
train_df=pd.read_csv('../input/titanic/train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [71]:
train_df.shape

(891, 12)

In [72]:
test_df=pd.read_csv('../input/titanic/test.csv')
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [73]:
test_df.shape

(418, 11)

 ### In my idea passengerid, name, Ticket are not important so I will delet all of them 

In [74]:
train_df.drop(['PassengerId', 'Name','Ticket'], axis=1,inplace=True)
test_df.drop(['PassengerId', 'Name','Ticket'], axis=1,inplace=True)



## Deal with missing value

In [75]:
train_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [76]:
#train_df.drop(['Cabin'], axis=1,inplace=True)

In [77]:
train_df['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [78]:
train_df['Cabin'].value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64

Because age is a nature thing we can fill that with mean and embarked with mode.

In [79]:
train_df['Age'].fillna((train_df['Age'].mean()), inplace=True)


train_df['Embarked'].fillna( method ='ffill', inplace = True)

train_df["Cabin"].fillna("G6", inplace = True)


In [80]:
train_df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

We calculate the cabib mising value rate

In [81]:
train_df.isnull().sum() / train_df.shape[0] * 100.00


Survived    0.0
Pclass      0.0
Sex         0.0
Age         0.0
SibSp       0.0
Parch       0.0
Fare        0.0
Cabin       0.0
Embarked    0.0
dtype: float64

Over 50% So it is better to eliminate this column too.

If we do the same for our Test data set:

In [82]:
test_df.isnull().sum()

Pclass        0
Sex           0
Age          86
SibSp         0
Parch         0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [83]:
test_df['Cabin'].value_counts()

B57 B59 B63 B66    3
B45                2
C89                2
C55 C57            2
A34                2
                  ..
E52                1
D30                1
E31                1
C62 C64            1
C105               1
Name: Cabin, Length: 76, dtype: int64

In [84]:
test_df.isnull().sum() / train_df.shape[0] * 100.00


Pclass       0.000000
Sex          0.000000
Age          9.652076
SibSp        0.000000
Parch        0.000000
Fare         0.112233
Cabin       36.700337
Embarked     0.000000
dtype: float64

In [85]:
test_df['Age'].fillna((test_df['Age'].mean()), inplace=True)

test_df['Embarked'].fillna((test_df['Embarked'].mode()), inplace=True)

test_df['Fare'].fillna((test_df['Fare'].mean()), inplace=True)
test_df["Cabin"].fillna("B63", inplace = True)


In [86]:
#test_df.drop(['Cabin'], axis=1,inplace=True)

In [87]:
test_df.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [88]:
train_df['Sex'].replace( 'female',1,inplace=True)
train_df['Sex'].replace( 'male',2,inplace=True)

In [89]:
test_df['Sex'].replace( 'female',1,inplace=True)
test_df['Sex'].replace('male',2,inplace=True)

In [90]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,2,22.0,1,0,7.2500,G6,S
1,1,1,1,38.0,1,0,71.2833,C85,C
2,1,3,1,26.0,0,0,7.9250,G6,S
3,1,1,1,35.0,1,0,53.1000,C123,S
4,0,3,2,35.0,0,0,8.0500,G6,S


In [91]:
train_df['Embarked'].unique()

array(['S', 'C', 'Q'], dtype=object)

In [92]:
train_df['Embarked'].replace( 'S',1,inplace=True)
train_df['Embarked'].replace( 'C',2,inplace=True)
train_df['Embarked'].replace( 'Q',3,inplace=True)

In [93]:
test_df['Embarked'].replace( 'S',1,inplace=True)
test_df['Embarked'].replace( 'C',2,inplace=True)
test_df['Embarked'].replace( 'Q',3,inplace=True)

In [94]:
 from sklearn import preprocessing 


In [95]:
    le = preprocessing.LabelEncoder()

In [96]:
train_df['Cabin']=le.fit_transform(train_df['Cabin'])

In [97]:
test_df['Cabin']=le.fit_transform(test_df['Cabin'])

In [98]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,2,22.0,1,0,7.2500,145,1
1,1,1,1,38.0,1,0,71.2833,81,2
2,1,3,1,26.0,0,0,7.9250,145,1
3,1,1,1,35.0,1,0,53.1000,55,1
4,0,3,2,35.0,0,0,8.0500,145,1


In [99]:
X, y = train_df.drop('Survived', axis = 1), train_df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [100]:
import imblearn

In [101]:
from imblearn.over_sampling import SMOTE

In [102]:
oversample = SMOTE()

In [103]:
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [104]:
scaler = StandardScaler()

In [105]:
scaler.fit_transform(X_train,X_test)

array([[ 0.92920087, -1.12337204, -0.18189205, ..., -0.46315365,
         0.4634281 , -0.56286591],
       [-0.26113063,  0.89017704,  1.73536071, ..., -0.43510224,
         0.4634281 , -0.56286591],
       [-0.26113063,  0.89017704, -0.18189205, ..., -0.42552804,
         0.4634281 , -0.56286591],
       ...,
       [-0.26113063, -1.12337204, -0.51185413, ..., -0.16349719,
         0.4634281 , -0.56286591],
       [-0.26113063, -1.12337204,  0.80136548, ...,  0.08343043,
         0.4634281 , -0.56286591],
       [-1.45146213, -1.12337204, -0.49838099, ...,  1.05595406,
        -2.38121972, -0.56286591]])

In [106]:
scaler.fit_transform(test_df)

array([[ 0.87348191,  0.75592895,  0.3349926 , ..., -0.49840706,
        -0.32690361,  2.24316522],
       [ 0.87348191, -1.32287566,  1.32553003, ..., -0.51327429,
        -0.32690361, -0.6778412 ],
       [-0.31581919,  0.75592895,  2.51417495, ..., -0.46508846,
        -0.32690361,  2.24316522],
       ...,
       [ 0.87348191,  0.75592895,  0.65196458, ..., -0.50879189,
        -0.32690361, -0.6778412 ],
       [ 0.87348191,  0.75592895,  0.        , ..., -0.4944482 ,
        -0.32690361, -0.6778412 ],
       [ 0.87348191,  0.75592895,  0.        , ..., -0.23790598,
        -0.32690361,  0.78266201]])

In [107]:
#from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


In [108]:
#space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
       # 'gamma': hp.uniform ('gamma', 1,9),
       # 'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
      #  'reg_lambda' : hp.uniform('reg_lambda', 0,1),
       # 'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
      #  'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
       # 'n_estimators': 180,
   #     'seed': 0
    #}

In [109]:
#def objective(space):
   # clf=xgb.XGBClassifier(
                  #  n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                  #  reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                 #   colsample_bytree=int(space['colsample_bytree']))
    
   # evaluation = [( X_train, y_train), ( X_test, y_test)]
    
   # clf.fit(X_train, y_train,
            #eval_set=evaluation, eval_metric="auc",
          #  early_stopping_rounds=10,verbose=False)
    

    #pred = clf.predict(X_test)
   # accuracy = accuracy_score(y_test, pred>0.5)
  #  print ("SCORE:", accuracy)
  #  return {'loss': -accuracy, 'status': STATUS_OK }

In [110]:
#trials = Trials()

#best_hyperparams = fmin(fn = objective,
                   #     space = space,
                    #    algo = tpe.suggest,
                     #   max_evals = 100,
                      #  trials = trials)

In [111]:
#print("The best hyperparameters are : ","\n")
#print(best_hyperparams)

In [112]:
#model =xgb. XGBClassifier(n_estimators=2000, colsample_bytree= 0.782912138847328,gamma= 1.2484661685986642,max_depth=15,
                    # min_child_weight=3.0, reg_alpha= 43.0,reg_lambda= 0.32889272247033874)
                     


In [113]:

#model.fit(X_train, y_train)

In [114]:
#expected_y  = y_test
#predicted_y = model.predict(X_test)

In [115]:
#print(metrics.classification_report(expected_y, predicted_y))
#print(metrics.confusion_matrix(expected_y, predicted_y))

In [116]:
#new_pred = model.predict(test_df)

In [117]:
#submission=pd.read_csv("../input/titanic/test.csv")

In [118]:
#submission['Survived'] = new_pred

In [119]:
#submission.head()

In [120]:
#submission.drop(['Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked'], axis=1,inplace=True)

In [121]:
#submission=submission.to_csv("submission.csv", index=False)

In [122]:
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [123]:
from sklearn.ensemble import RandomForestClassifier

In [124]:
rfc = RandomForestClassifier()


In [125]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [126]:
#grid_space={'max_depth':[3,5,10,None],
             #'n_estimators':[10,100,200],
            # 'max_features':[1,3,5,7],
          #   'min_samples_leaf':[1,2,3],
           #   'min_samples_split':[1,2,3]
         # }

In [127]:
#grid = GridSearchCV(rfc,param_grid=grid_space,cv=3,scoring='accuracy')
#model_grid = grid.fit(X_train,y_train)

In [128]:
#print('Best hyperparameters are: '+str(model_grid.best_params_))
#print('Best score is: '+str(model_grid.best_score_))

In [129]:
rfc = RandomForestClassifier(max_depth=5, max_features=7,min_samples_leaf=1, min_samples_split=3,n_estimators= 100 )


In [130]:
model = rfc.fit(X_train,y_train)

In [131]:
expected_y  = y_test
predicted_y = model.predict(X_test)

In [132]:
print(metrics.classification_report(expected_y, predicted_y))
print(metrics.confusion_matrix(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.81      0.88      0.85       100
           1       0.83      0.75      0.79        79

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.82       179
weighted avg       0.82      0.82      0.82       179

[[88 12]
 [20 59]]


In [133]:
new_pred = model.predict(test_df)

In [134]:
submission=pd.read_csv("../input/titanic/test.csv")

In [135]:
submission['Survived'] = new_pred

In [136]:
submission.drop(['Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked'], axis=1,inplace=True)

In [137]:
submission=submission.to_csv("submission5.csv", index=False)